# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
Let's consider the following LCIA file (saved under `lcia_example_2.json`):

In [2]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "CH" # now, we want to specify the location of the consumer
    },
    "value": 1.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "FR" # now, we want to specify the location of the consumer
    },
    "value": 2.0
  },
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process",
      "location": "RER" # now, we want to specify the location of the consumer
    },
    "value": 3.0
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'CH'},
  'value': 1.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'FR'},
  'value': 2.0},
 {'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process', 'location': 'RER'},
  'value': 3.0}]

In [2]:
# activate the bw project
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, RU, None)

In [3]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 209.01it/s]


3.5226391576625655

### Generate dataframe of characterization factors used

The `generate_cf_table` method generates a dataframe of the characterization factors used in the calculation. One can see the characterization factors used for each exchange in the system.

In [4]:
df = LCA.generate_cf_table()

In [5]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, in air","(natural resource, in air)","rye production, organic","straw, organic",CH,7.607581e-09,1.0,7.607581e-09
1,"Carbon dioxide, in air","(natural resource, in air)","fruit tree seedling production, for planting","fruit tree seedling, for planting",CH,2.326311e-13,1.0,2.326311e-13
2,"Carbon dioxide, in air","(natural resource, in air)",beet sugar production,sugar beet pulp,CH,1.704264e-28,1.0,1.704264e-28
3,"Carbon dioxide, in air","(natural resource, in air)","strawberry production, in unheated greenhouse",strawberry,CH,4.306825e-30,1.0,4.306825e-30
4,"Carbon dioxide, in air","(natural resource, in air)","potato seed production, Swiss integrated produ...","potato seed, Swiss integrated production, at farm",CH,4.491381e-08,1.0,4.491381e-08
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, non-fossil","(air,)","containerboard production, linerboard, kraftliner","containerboard, linerboard",RER,5.869426e-09,3.0,1.760828e-08
1206,"Carbon dioxide, non-fossil","(air,)",anthraquinone production,anthraquinone,RER,8.717202e-11,3.0,2.615161e-10
1207,"Carbon dioxide, non-fossil","(air,)",latex production,latex,RER,1.891544e-13,3.0,5.674632e-13
1208,"Carbon dioxide, non-fossil","(air,)","particleboard production, cement bonded","particleboard, cement bonded",RER,2.081747e-12,3.0,6.245240e-12


As expected, only CH, FR and RER-based consumers have been considered in the calculation.

In [6]:
df.groupby("consumer location")["CF"].mean()

consumer location
CH     1.0
FR     2.0
RER    3.0
Name: CF, dtype: float64

But `statistics()` tells us that many exchanges have been ignored.

In [7]:
LCA.statistics()

+----------------------+--------------------------------------------+
|       Activity       | natural gas, high pressure, import from TT |
|     Method name      |             ('some', 'method')             |
|      Data file       |               lcia_example_2               |
| Unique CFs in method |                     3                      |
|   Unique CFs used    |                     3                      |
|  Exc. characterized  |                    1210                    |
| Exc. uncharacterized |                   334699                   |
+----------------------+--------------------------------------------+


We can extend the application of CFs to other exchanges, using the helping functions: `map_aggregate_locations`, `map_disaggregate_location`, `map_dynamic_locations` and `map_remaining_locations_to_global`.

* `map_aggregate_locations` maps exchanges with aggregate locations (e.g., "RER", RLA", etc.) to weighted-average CFs.
* `map_disaggregate_location` maps exchanges with locations that belong to aggregate locations ("CA-QC") to CFs of aggregate locations (e.g., "CA").
* `map_dynamic_locations` maps exchanges with dynamic locations (e.g., "RoW", "RoW") to CFs calculated as the weighted-average of all locations minus those excluded by dynamic locations' perimeter (e.g., "GLO minus CH").
* `map_remaining_locations_to_global` maps exchanges with remaining locations to the global CFs.

In [3]:
LCA = EdgeLCIA(
    demand={act: 1},
    method=("some", "method"),
    filepath="lcia_example_2.json"
)
LCA.lci()

LCA.map_exchanges()

LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_contained_locations()
LCA.map_remaining_locations_to_global()

LCA.evaluate_cfs()
LCA.lcia()
LCA.score

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 243.64it/s]

Handling static regions...



Processing remaining static edges (pass 2): 100%|██████████| 96785/96785 [00:00<00:00, 573761.07it/s]


Handling dynamic regions...


Processing remaining dynamic edges (pass 2): 100%|██████████| 84848/84848 [00:00<00:00, 465630.12it/s]


Handling contained locations...
Handling remaining exchanges...


Processing remaining global edges (pass 2): 100%|██████████| 333196/333196 [00:01<00:00, 318460.49it/s]


0.17953711642085313

In [9]:
LCA.statistics()

+----------------------+--------------------------------------------+
|       Activity       | natural gas, high pressure, import from TT |
|     Method name      |             ('some', 'method')             |
|      Data file       |               lcia_example_2               |
| Unique CFs in method |                     3                      |
|   Unique CFs used    |                     3                      |
|  Exc. characterized  |                    1210                    |
| Exc. uncharacterized |                   334699                   |
+----------------------+--------------------------------------------+


In [10]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

In [4]:
df = LCA.generate_cf_table()

In [12]:
df

,supplier name,supplier categories,consumer name,consumer reference product,consumer location,amount,CF,impact
0,"Carbon dioxide, in air","(natural resource, in air)","rye production, organic","straw, organic",CH,7.607581e-09,1.0,7.607581e-09
1,"Carbon dioxide, in air","(natural resource, in air)","fruit tree seedling production, for planting","fruit tree seedling, for planting",CH,2.326311e-13,1.0,2.326311e-13
2,"Carbon dioxide, in air","(natural resource, in air)",beet sugar production,sugar beet pulp,CH,1.704264e-28,1.0,1.704264e-28
3,"Carbon dioxide, in air","(natural resource, in air)","strawberry production, in unheated greenhouse",strawberry,CH,4.306825e-30,1.0,4.306825e-30
4,"Carbon dioxide, in air","(natural resource, in air)","potato seed production, Swiss integrated produ...","potato seed, Swiss integrated production, at farm",CH,4.491381e-08,1.0,4.491381e-08
...,...,...,...,...,...,...,...,...
1205,"Carbon dioxide, non-fossil","(air,)","containerboard production, linerboard, kraftliner","containerboard, linerboard",RER,5.869426e-09,3.0,1.760828e-08
1206,"Carbon dioxide, non-fossil","(air,)",anthraquinone production,anthraquinone,RER,8.717202e-11,3.0,2.615161e-10
1207,"Carbon dioxide, non-fossil","(air,)",latex production,latex,RER,1.891544e-13,3.0,5.674632e-13
1208,"Carbon dioxide, non-fossil","(air,)","particleboard production, cement bonded","particleboard, cement bonded",RER,2.081747e-12,3.0,6.245240e-12


We can check the weights used:

In [5]:
LCA.weights

{'CH': 8921981, 'FR': 66548530, 'RER': 0}

By default, if weights are not provided in the consumer sections of LCIA file, population is used. The other option is GDP. To be specified when calling EdgeLICA().

And we can see the weighted-average CFs used:

In [5]:
df.groupby("consumer location")["CF"].mean()

consumer location
AE        1.881782
AR        1.881782
AT        1.881782
AU        1.881782
AU-NSW    1.881782
            ...   
VE        1.881782
VN        1.881782
WEU       1.881782
ZA        1.881782
ZM        1.881782
Name: CF, Length: 227, dtype: float64

We have pre-generated LCIA files for specific methods (AWARE, ImpactWorld+, etc.).
You can get a list like so:

In [37]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

Check the other notebooks to see how to use them.